In [ ]:
#Executive $cript
#this should have three phases 1) dependencies and credit, 2) definitinon of FUNS and 3) EXECUTION

In [1]:
#DEPENDENCIES AND CREDIT AND BASIC EXECUTIVE SETTINGS
#Credit:
# Solid_Horse_5896 (altough it prob wouldn't recognize it anymore)


# https://www.reddit.com/r/pushshift/comments/m6difm/how_to_download_all_the_comments_of_a_subreddit/
#respective: https://colab.research.google.com/drive/1eqpTm8BW4k9RiZPcQJO2so7Jny7kwAXn?usp=sharing


#! python3
!pip install praw
!pip install psaw
!pip install pmaw
import praw
import pandas as pd
import datetime as dt
import time
import math
from IPython.display import clear_output
import timeit
import csv
import numpy as np
import pmaw
import statistics as st
from pmaw import PushshiftAPI
from itertools import chain
import random

api = PushshiftAPI() #do not change

###########################################################
subreddit = 'gendercritical' #list the subreddit name here
###########################################################

#######################################################################
botlist= "E:/Claudio/Documents/Masterarbeit_Daten/botlist.csv"#Write your botlist-directory here
##############################################################

#######################################################################
datapath= "E:/Claudio/Documents/Masterarbeit_Daten/reddit_data/%s.csv"#Write your data-directory here  (keep the ending "%s.csv")
##############################################################


#important for function "get_time_of_the_banisment"
Ban_Days= 60*24*60*60 #how big steps you want to leap backwards in finding the last submission of a banned sub (is important for function get_time_of_the_banishment)
NotBanned = 60*60*24*2 #how many days should be checked if a sub is not banned (min 1 submission needs to be found)
last_point= 1619780212 #This is the starting point of the scripts it will only go back in time from there (30.3.21)

#important for the function "get_all_comms", "get_all_subs", "get_all_comms_per_auth" and "get_all_subs_per_auth"
Days= 31*24*60*60 #this controls how much back a user has commented to count it as active and study it
Follow_Days = 31*24*60*60  # This controls how much back or forth the user will be followed after the ban

#important for bot-list
manual_bad_accounts=["[deleted]", "nsfw_celbs", "AutoModerator", "RemindMeBot","SaveVideo","savevideobot","converter-bot","epic_gamer_4268","WaitingToBeTriggered","dadbot_3000","SpunkyDred","remindditbot"] #add all accounts which you don't want, because they are bots or spammers or index for deleted account.


In [6]:
replacer=datapath

name_sub=(replacer%(subreddit+"selected_authors"+"SUBMISSIONS"+"BEFORE_BAN"+"CONTROL"))
print(name_sub)
print(datapath%(subreddit+"selected_authors"+"SUBMISSIONS"+"BEFORE_BAN"+"CONTROL"))

E:/Claudio/Documents/Masterarbeit_Daten/reddit_data/gendercriticalselected_authorsSUBMISSIONSBEFORE_BANCONTROL.csv
E:/Claudio/Documents/Masterarbeit_Daten/reddit_data/gendercriticalselected_authorsSUBMISSIONSBEFORE_BANCONTROL.csv


In [2]:
#THIS PART IS THE DEFINTION OF FUNs

#PART ONE: GETTING THE EPOCH-TIME OF THE BANNING OF THE SUBREDDIT**************************************************************************

#Time of the banishment


#This scripts starts end of april and looks for submissions. If it is banned, than no submissions will be found and
# the script will go further back in time. If it finds something in the first 2 Days, the subreddit is not banned. So you could check for that.
# reddit only began with banning in 2015, so if the script looks further back somthing fails and a file is created with
# "TOOLATE" in the name, indicating problems. 
# What we really want is a .csv-file wich is always names after the subreddit+"BANISHMENT"

def get_time_of_the_banishment(last_point,subreddit,limit = 100000):
  completed = '  after date            # of subs        time\n'
  start_time = timeit.default_timer()
  sub_df = []
  before = last_point
  after = last_point-Ban_Days 

  subs=api.search_submissions(subreddit=subreddit, limit=limit, before=before, after =before-NotBanned)

  sub_df = pd.DataFrame(subs)
  if len(sub_df)>0:
        
        name_sub=((datapath%(subreddit+"NOTBANNED")))
        sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  else:

      while len(sub_df)<1:  



        subs=api.search_submissions(subreddit=subreddit, limit=limit, before=before, after =after)

        sub_df = pd.DataFrame(subs)

        name_sub=((datapath%(subreddit+"BANISHMENT")))
        sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
        before = before-Days
        after = before-Days
        if after<1420074061:
            name_sub=((datapath%(subreddit+"TOOLATE")))
            sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
            print("iwas here, something went terribly wrong.")
            if 1==1:
                break
            

            


      #clear_output(wait=True)
      end_time=timeit.default_timer()
      sub_len = len(sub_df)
  

    
    
    

   
    
  print(completed)



#PART TWO: FINDING ALL THE ACTIVE USERS OF A SUB BEFORE THE BAN**************************************************************************

#this function goes from a variable point in time back a variable amount of time and saves all comments in a sub
#and saves it as a .csv, with the name subreddit+"COMMENT".csv



def get_all_coms(after_date,before_date,subreddit,time_span = Days,limit = 1000000):
  
  start_time = timeit.default_timer()


  before = before_date
  after = before - time_span
        

  subs=api.search_comments(subreddit=subreddit, limit=limit, before=before, after=after)

  sub_df = pd.DataFrame(subs)

  name_sub=((datapath%(subreddit+"COMMENT")))
  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  #clear_output(wait=True)   
    
  elapsed = timeit.default_timer() - start_time  
    

   
    
  print(elapsed/60, "get_all_comms")
  #if sub_len == 100000:
   #     print("theres too much Data, Information has been lost !!!11!11 comments_user_after************************************************************************************************************")




#now the same, but for submissions:

def get_all_subs(after_date,before_date,subreddit,time_span = Days,limit = 100000):

  start_time = timeit.default_timer()

  before = before_date
  after = before - time_span
        

  subs=api.search_submissions(subreddit=subreddit, limit=limit, before=before, after=after)

  sub_df = pd.DataFrame(subs)

  name_sub=((datapath%(subreddit+"SUBMISSION")))
  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  #clear_output(wait=True)
  end_time=timeit.default_timer()
    
  elapsed = timeit.default_timer() - start_time  
  
    
    

   
    
  print(elapsed/60, "GET_ALL_SUBS")


#PART THREE: FOLLOWIN THE ACTIVE USERS**************************************************************************

# of all the identified active users of the banned subreddit 
# we want alle the comments and submissions 
# for 31 Days before the ban
# and after the ban.


#First the followed users' submissions before the banning

def get_all_subs_per_auth_before(after_date,before_date,authors,time_span = Follow_Days,limit = 10000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date
  after = before - time_span
  subs = list(range( len(authors)))
#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.
 
  for i in subs:
        subs[i]=api.search_submissions(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%(subreddit+"selected_authors"+"SUBMISSIONS"+"BEFORE_BAN"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  #clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    
    
    
  print(completed, "get_all_subs_per_auth_before_finished_above")
  
   
        
  

#Secondly the followed users' submissions after the ban
#its basically the same
def get_all_subs_per_auth_after(after_date,before_date,authors,time_span = Follow_Days,limit = 10000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date + Follow_Days+1 #plus one because we don't want overlap
  after = before - time_span
  subs = list(range( len(authors)))
#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.
 
  for i in subs:
        subs[i]=api.search_submissions(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%(subreddit+"selected_authors"+"SUBMISSIONS"+"AFTER_BAN"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  #clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    #completed = (%s%s %(str(dt.datetime.fromtimestamp(before))+'   ' +str(len(sub_df)) +'                     '+str(np.round((end_time-start_time)/60,2))+'\n'
    
    
  print(completed, "get_all_subs_per_auth_after_finished_above")
  
        

#Now the same, but for comments

def get_all_coms_per_auth_before(after_date,before_date,authors,time_span = Follow_Days,limit = 10000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date
  after = before - time_span
  subs = list(range( len(authors)))
#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.
 
  for i in subs:
        subs[i]=api.search_comments(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%(subreddit+"selected_authors"+"COMMENTS"+"BEFORE_BAN"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  #clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    
    
    
  print(completed,"get_all_coms_per_auth_before_finished_above")
  
#Now the again for comments before, but this time for big subreddits. 

def get_all_coms_per_auth_before_big1(after_date,before_date,authors,time_span = Follow_Days,limit = 10000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date
  after = before - time_span
  subs = list(range( len(authors)//2))
#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.
 
  for i in subs:
        subs[i]=api.search_comments(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%(subreddit+"selected_authors"+"COMMENTS"+"BEFORE_BAN1"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  #clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    
    
    
  print(completed,"get_all_coms_per_auth_before_finished_above")

def get_all_coms_per_auth_before_big2(after_date,before_date,authors,time_span = Follow_Days,limit = 10000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date
  after = before - time_span
  position= len(authors)//2
  subs = list(range(len(authors)-position))

#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.

  authors=authors[position:]
  for i in subs:
        subs[i]=api.search_comments(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%(subreddit+"selected_authors"+"COMMENTS"+"BEFORE_BAN2"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  #clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    
    
    
  print(completed,"get_all_coms_per_auth_before_finished_above")

#now comments after the ban:

def get_all_coms_per_auth_after(after_date,before_date,authors,time_span = Follow_Days,limit = 10000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date + Follow_Days+1 #plus one because we don't want overlap
  after = before - time_span
  subs = list(range( len(authors)))
#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.
 
  for i in subs:
        subs[i]=api.search_comments(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%(subreddit+"selected_authors"+"COMMENTS"+"AFTER_BAN"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  #clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    #completed = (%s%s %(str(dt.datetime.fromtimestamp(before))+'   ' +str(len(sub_df)) +'                     '+str(np.round((end_time-start_time)/60,2))+'\n'
    
    
  print(completed, "get_all_coms_per_auth_after_finished")
  
 ##########################################   
#The next ones are for the control-group:##
###########################################

def get_all_subs_per_auth_before_control(after_date,before_date,authors,time_span = Follow_Days,limit = 10000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date
  after = before - time_span
  subs = list(range( len(authors)))
#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.
 
  for i in subs:
        subs[i]=api.search_submissions(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%(subreddit+"selected_authors"+"SUBMISSIONS"+"BEFORE_BAN"+"CONTROL"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  #clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    
    
    
  print(completed, "get_all_subs_per_auth_before_ control finished_above")
  
   
        
  

#Secondly the followed users' submissions after the ban
#its basically the same
def get_all_subs_per_auth_after_control(after_date,before_date,authors,time_span = Follow_Days,limit = 10000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date + Follow_Days+1 #plus one because we don't want overlap
  after = before - time_span
  subs = list(range( len(authors)))
#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.
 
  for i in subs:
        subs[i]=api.search_submissions(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%(subreddit+"selected_authors"+"SUBMISSIONS"+"AFTER_BAN"+"CONTROL"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  #clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    #completed = (%s%s %(str(dt.datetime.fromtimestamp(before))+'   ' +str(len(sub_df)) +'                     '+str(np.round((end_time-start_time)/60,2))+'\n'
    
    
  print(completed, "get_all_subs_per_auth_after_control finished_above")
  
#to not overwrite my files I have to re-name the complete download of a sub to get the authors for the control-group


def get_all_subs_control(after_date,before_date,subreddit,time_span = Days,limit = 100000):

  start_time = timeit.default_timer()

  before = before_date
  after = before - time_span
        

  subs=api.search_submissions(subreddit=subreddit, limit=limit, before=before, after=after)

  sub_df = pd.DataFrame(subs)

  name_sub=((datapath%(subreddit+"SUBMISSION_CONTROLGROUP")))
  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  #clear_output(wait=True)
  end_time=timeit.default_timer()
    
  elapsed = timeit.default_timer() - start_time  
  
    
    

   
    
  print(elapsed/60, "GET_ALL_SUBS")



##### Below here trash: checking if control subreddit is right


def get_all_subs_control_of_control(after_date,before_date,subreddit,time_span = Days,limit = 100000):

  start_time = timeit.default_timer()

  before = before_date
  after = before - time_span
        

  subs=api.search_submissions(subreddit=subreddit, limit=limit, before=before, after=after)

  sub_df = pd.DataFrame(subs)

  name_sub=((datapath%(subreddit+"SUBMISSION_control_of_control")))
  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  #clear_output(wait=True)
  end_time=timeit.default_timer()
    
  elapsed = timeit.default_timer() - start_time  
  
    
    

   
    
  print(elapsed/60, "GET_ALL_SUBS")



In [ ]:
#The last part of the python Triptychon: Execution

#1.1 Getting the time when the sub was banned.

get_time_of_the_banishment(last_point,subreddit)

#1.2 extracting the banishment-time

df = pd.read_csv((datapath%(subreddit+"BANISHMENT")))
data=df["created_utc"].sort_values(ascending=False).reset_index(drop=True)
banishment_epoch=data[0]

#2. Finding all active Users of the sub****************************************
#2.1 The Users in the Submissions

get_all_subs(banishment_epoch-Days, banishment_epoch, subreddit, Days)

#2.2 The Users in the Comments

get_all_coms(banishment_epoch-Days, banishment_epoch, subreddit, Days)

#2.3 Creating botlist and other bad_accountsnames

#Getting Botlist
bad_accounts_raw = pd.read_csv(botlist)

a=0
bad_accounts=[]
#yes, this is the best way I came up with of converting a df into a list
for i in range(len(bad_accounts_raw)):
    bad_accounts.append(bad_accounts_raw.iloc[a,0])
    a=a+1

#here we combine it with the manual bad account list above
bad_accounts.extend(manual_bad_accounts)
    

#Creating List of all unique authors, dropping bad_accounts


df = pd.read_csv(datapath %(subreddit+"SUBMISSION"))

df = df[~df.author.isin(bad_accounts)] #this removes all bad accounts contained in our list
    
    
df2 = pd.read_csv(datapath %(subreddit+"COMMENT"))

df2 = df2[~df2.author.isin(bad_accounts)] #see above


df= df.append(df2)#combining our lists (theres plenty of overlap though, most of the submissers also commented)


data= df.groupby(["author"]).size().reset_index(drop=False) #we aggregate all the activity of the accounts and count them.
#This creates a df, which has a axis in which every account is contained and unique


author = list(data.iloc[:,0])#But we actually want a list of all unique authors

#3. Following our identified Users**********************************************************
#3.1 comments before ban

get_all_coms_per_auth_before(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 10000)

#3.2 comments after ban

get_all_coms_per_auth_after(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 10000)

#3.3 Submissions before ban

get_all_subs_per_auth_before(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 10000)


#3.4 Submissions after ban


get_all_subs_per_auth_after(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 10000)

print("If This shows, you really did it Claudio!!!")
    

0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
580 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 16 - Batches: 3 - Items Remaining: 0
  after date            # of subs        time

3152 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 40 - Batches: 4 - Items Remaining: 0
0.6958613316666664 GET_ALL_SUBS
58171 results available in Pushshift
Checkpoint:: Success Rate: 92.00% - Requests: 100 - Batches: 10 - Items Remaining: 48971
Checkpoint:: Success Rate: 89.50% - Requests: 200 - Batches: 20 - Items Remaining: 40382
Checkpoint:: Success Rate: 89.67% - Requests: 300 - Batches: 30 - Items Remaining: 31471
Checkpoint:: Success Rate: 89.00% - Requests: 400 - Batche

3 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 5 - Batches: 1 - Items Remaining: 0
218 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
8 results available in Pushshift
Total:: Success Rate: 39.13% - Requests: 23 - Batches: 4 - Items Remaining: 0
93 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
21 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
124 results available in Pushshift
Total:: Success Rate: 55.56% - Requests: 18 - Batches: 5 - Items Remaining: 0
5 results available in Pushshift
Total:: Success Rate: 75.00% - Requests: 8 - Batches: 1 - Items Remaining: 0
48 results available in Pushshift
Total:: Success Rate: 83.33% - Requests: 12 - Batches: 3 - Items Remaining: 0
386 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Re

4 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 8 - Batches: 1 - Items Remaining: 0
14 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
10 results available in Pushshift
Total:: Success Rate: 44.44% - Requests: 18 - Batches: 2 - Items Remaining: 0
39 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
18 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 8 - Batches: 1 - Items Remaining: 0
27 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
3 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 6 - Batches: 1 - Items Remaining: 0
22 results available in Pushshift
Total:: Success Rate: 55.56% - Requests: 18 - Batches: 4 - Items Remaining: 0
56 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 9 - Batches: 1 - Items Rema

Total:: Success Rate: 100.00% - Requests: 5 - Batches: 1 - Items Remaining: 0
546 results available in Pushshift
Total:: Success Rate: 91.67% - Requests: 12 - Batches: 3 - Items Remaining: 0
206 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
48 results available in Pushshift
Total:: Success Rate: 71.43% - Requests: 14 - Batches: 2 - Items Remaining: 0
1 results available in Pushshift
Total:: Success Rate: 22.22% - Requests: 9 - Batches: 1 - Items Remaining: 0
7 results available in Pushshift
Total:: Success Rate: 83.33% - Requests: 12 - Batches: 2 - Items Remaining: 0
178 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
142 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
19 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 9 - Batches: 1 - Items Remaining: 0
17 results available i

2 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
26 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 9 - Batches: 1 - Items Remaining: 0
138 results available in Pushshift
Total:: Success Rate: 58.82% - Requests: 17 - Batches: 2 - Items Remaining: 0
37 results available in Pushshift
Total:: Success Rate: 60.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
181 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
48 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 6 - Batches: 1 - Items Remaining: 0
14 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
4 results available in Pushshift
Total:: Success Rate: 55.56% - Requests: 18 - Batches: 2 - Items Remaining: 0
613 results available in Pushshift
Total:: Success Rate: 75.00% - Requests: 16 - Batches: 3 - Items R

...

22 results available in Pushshift
Total:: Success Rate: 81.82% - Requests: 11 - Batches: 2 - Items Remaining: 0
0 results available in Pushshift
0 results available in Pushshift
2 results available in Pushshift
Total:: Success Rate: 41.67% - Requests: 12 - Batches: 2 - Items Remaining: 0
0 results available in Pushshift
11 results available in Pushshift
Total:: Success Rate: 41.67% - Requests: 24 - Batches: 4 - Items Remaining: 0
0 results available in Pushshift
42 results available in Pushshift
Total:: Success Rate: 50.00% - Requests: 20 - Batches: 4 - Items Remaining: 0
0 results available in Pushshift
4 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 6 - Batches: 1 - Items Remaining: 0
0 results available in Pushshift
5 results available in Pushshift
Total:: Success Rate: 43.48% - Requests: 23 - Batches: 3 - Items Remaining: 0
0 results available in Pushshift
0 results available in Pushshift
17 results available in Pushshift
Total:: Success Rate: 50.00% - Re

3 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
1 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 4 - Batches: 1 - Items Remaining: 0
0 results available in Pushshift
1 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 3 - Batches: 1 - Items Remaining: 0
2 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 2 - Batches: 1 - Items Remaining: 0
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
1 results available in Pushshift
Total:: Success Rate: 46.67% - Requests: 15 - Batches: 2 - Items Remaining: 0
4 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 8 - Batches: 1 - Items Remaining: 0
4 results available in Pushshift
Total:: Success Rate: 100.00% - Requ

0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
1 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 2 - Batches: 1 - Items Remaining: 0
3 results available in Pushshift
Total:: Success Rate: 71.43% - Requests: 14 - Batches: 2 - Items Remaining: 0
0 results available in Pushshift
0 results available in Pushshift
1 results available in Pushshift
Total:: Success Rate: 38.46% - Requests: 13 - Batches: 2 - Items Remaining: 0
0 results available in Pushshift
0 results available in Pushshift
7128 results available in Pushshift
Checkpoint:: Success Rate: 89.25% - Requests: 93 - Batches: 10 - Items Remaining: 4
Total:: Success Rate: 89.36% - Requests: 94 - Batches: 11 - Items Remaining: 0
1 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1 - Items Remaini

Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
5 results available in Pushshift
Total:: Success Rate: 66.67% - Requests: 9 - Batches: 1 - Items Remaining: 0
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
1 results available in Pushshift
Total:: Success Rate: 42.11% - Requests: 19 - Batches: 3 - Items Remaining: 0
0 results available in Pushshift
5 results available in Pushshift
Total:: Success Rate: 50.00% - Requests: 20 - Batches: 6 - Items Remaining: 0
0 results available in Pushshift
0 results available in Pushshift
2 results available in Pushshift
Total:: Success Rate: 60.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
0 results available in Pushshift
3 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 9 - Batches: 1 - Items Remaining: 0
0 results available in Pushshift
3 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 2 - Batches: 1 - Items Remai

8 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 7 - Batches: 1 - Items Remaining: 0
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
9 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
0 results available in Pushshift
0 results available in Pushshift
0 results available in Pushshift
4 results available in Pushshift
Total:: Success Rate: 62.50% - Requests: 16 - Batches: 3 - Items Remaining: 0
11 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
22 results available in Pushshift
Total:: Success Rate: 90.91% - Requests: 11 - Batches: 2 - Items Remaining: 0
7 results available in Pushshift
Total:: Success Rate: 43.75% - Requests: 16 -

In [20]:
#If you would get an error, that memory is low while downloading the comments before of the accounts
#Chances are, that you there are too many comments. As a solution, run this code below
# that means, instead of running "get_all_coms_per_auth_before" your run "get_all_coms_per_auth_before_big1" and get_all_coms_per_auth_before_big2 



#1.2 extracting the banishment-time

df = pd.read_csv((datapath%(subreddit+"BANISHMENT")))
data=df["created_utc"].sort_values(ascending=False).reset_index(drop=True)
banishment_epoch=data[0]



#Getting Botlist
bad_accounts_raw = pd.read_csv(botlist)

a=0
bad_accounts=[]
#yes, this is the best way I came up with of converting a df into a list
for i in range(len(bad_accounts_raw)):
    bad_accounts.append(bad_accounts_raw.iloc[a,0])
    a=a+1

#here we combine it with the manual bad account list above
bad_accounts.extend(manual_bad_accounts)
    

#Creating List of all unique authors, dropping bad_accounts


df = pd.read_csv(datapath %(subreddit+"SUBMISSION"))

df = df[~df.author.isin(bad_accounts)] #this removes all bad accounts contained in our list
    
    
df2 = pd.read_csv(datapath %(subreddit+"COMMENT"))

df2 = df2[~df2.author.isin(bad_accounts)] #see above

df= df.append(df2)#combining our lists (theres plenty of overlap though, most of the submissers also commented)


data= df.groupby(["author"]).size().reset_index(drop=False) #we aggregate all the activity of the accounts and count them.
#This creates a df, which has a axis in which every account is contained and unique


author = list(data.iloc[:,0])#But we actually want a list of all unique authors


#3. Following our identified Users**********************************************************
#3.1 comments before ban

#get_all_coms_per_auth_before(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 10000)
get_all_coms_per_auth_before_big1(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 10000)
get_all_coms_per_auth_before_big2(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 10000)

#3.2 comments after ban

get_all_coms_per_auth_after(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 100000)

#3.3 Submissions before ban

get_all_subs_per_auth_before(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 100000)


#3.4 Submissions after ban

get_all_subs_per_auth_after(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 10000)

print("If This shows, you really did it Claudio!!!")


39 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 6 - Batches: 1 - Items Remaining: 0
28 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
110 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
39 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
11 results available in Pushshift
Total:: Success Rate: 66.67% - Requests: 9 - Batches: 1 - Items Remaining: 0
608 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 11 - Batches: 2 - Items Remaining: 0
29 results available in Pushshift
Total:: Success Rate: 66.67% - Requests: 15 - Batches: 3 - Items Remaining: 0
28 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
65 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Item

333 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
15 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 7 - Batches: 1 - Items Remaining: 0
56 results available in Pushshift
Total:: Success Rate: 64.29% - Requests: 14 - Batches: 2 - Items Remaining: 0
1 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 9 - Batches: 1 - Items Remaining: 0
6 results available in Pushshift
Total:: Success Rate: 81.82% - Requests: 11 - Batches: 2 - Items Remaining: 0
17 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
1 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 5 - Batches: 1 - Items Remaining: 0
42 results available in Pushshift
Total:: Success Rate: 52.63% - Requests: 19 - Batches: 2 - Items Remaining: 0
31 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Rema

5 results available in Pushshift
Total:: Success Rate: 66.67% - Requests: 9 - Batches: 1 - Items Remaining: 0
0 results available in Pushshift
15 results available in Pushshift
Total:: Success Rate: 31.25% - Requests: 32 - Batches: 4 - Items Remaining: 0
38 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
0 results available in Pushshift
25 results available in Pushshift
Total:: Success Rate: 90.91% - Requests: 11 - Batches: 2 - Items Remaining: 0
19 results available in Pushshift
Total:: Success Rate: 76.92% - Requests: 13 - Batches: 2 - Items Remaining: 0
198 results available in Pushshift
Total:: Success Rate: 37.04% - Requests: 27 - Batches: 5 - Items Remaining: 0
4 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 8 - Batches: 1 - Items Remaining: 0
8 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 8 - Batches: 1 - Items Remaining: 0
3 results available in Pushshift
Total::

Total:: Success Rate: 45.45% - Requests: 22 - Batches: 4 - Items Remaining: 0
33 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
6 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
105 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 6 - Batches: 1 - Items Remaining: 0
11 results available in Pushshift
Total:: Success Rate: 75.00% - Requests: 12 - Batches: 2 - Items Remaining: 0
14 results available in Pushshift
Total:: Success Rate: 35.00% - Requests: 20 - Batches: 3 - Items Remaining: 0
54 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
52 results available in Pushshift
Total:: Success Rate: 60.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
81 results available in Pushshift
Total:: Success Rate: 83.33% - Requests: 12 - Batches: 2 - Items Remaining: 0
2 results available in 

9 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
8 results available in Pushshift
Total:: Success Rate: 25.00% - Requests: 24 - Batches: 3 - Items Remaining: 0
47 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
10 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
3 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 4 - Batches: 1 - Items Remaining: 0
6 results available in Pushshift
Total:: Success Rate: 25.93% - Requests: 27 - Batches: 4 - Items Remaining: 0
1 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 4 - Batches: 1 - Items Remaining: 0
22 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
104 results available in Pushshift
Total:: Success Rate: 76.92% - Requests: 13 - Batches: 2 - Items Remai

...

Total:: Success Rate: 100.00% - Requests: 8 - Batches: 1 - Items Remaining: 0
103 results available in Pushshift
Total:: Success Rate: 90.91% - Requests: 11 - Batches: 2 - Items Remaining: 0
1 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1 - Items Remaining: 0
135 results available in Pushshift
Total:: Success Rate: 76.92% - Requests: 13 - Batches: 2 - Items Remaining: 0
4 results available in Pushshift
Total:: Success Rate: 57.14% - Requests: 7 - Batches: 1 - Items Remaining: 0
496 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
161 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
1 results available in Pushshift
Total:: Success Rate: 28.57% - Requests: 14 - Batches: 2 - Items Remaining: 0
6 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
59 results available in 

54 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
4 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
9 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 3 - Batches: 1 - Items Remaining: 0
152 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
1 results available in Pushshift
Total:: Success Rate: 45.45% - Requests: 11 - Batches: 2 - Items Remaining: 0
9 results available in Pushshift
Total:: Success Rate: 90.91% - Requests: 11 - Batches: 2 - Items Remaining: 0
1 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 3 - Batches: 1 - Items Remaining: 0
3 results available in Pushshift
Total:: Success Rate: 90.91% - Requests: 11 - Batches: 2 - Items Remaining: 0
5 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 9 - Batches: 1 - Items Remainin

89 results available in Pushshift
Total:: Success Rate: 39.13% - Requests: 23 - Batches: 6 - Items Remaining: 0
4 results available in Pushshift
Total:: Success Rate: 75.00% - Requests: 4 - Batches: 1 - Items Remaining: 0
5 results available in Pushshift
Total:: Success Rate: 87.50% - Requests: 8 - Batches: 1 - Items Remaining: 0
11 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
270 results available in Pushshift
Total:: Success Rate: 50.00% - Requests: 20 - Batches: 6 - Items Remaining: 0
0 results available in Pushshift
3 results available in Pushshift
Total:: Success Rate: 52.63% - Requests: 19 - Batches: 2 - Items Remaining: 0
41 results available in Pushshift
Total:: Success Rate: 87.50% - Requests: 8 - Batches: 1 - Items Remaining: 0
23 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 8 - Batches: 1 - Items Remaining: 0
75 results available in Pushshift
Checkpoint:: Success Rate: 27.78% - Reque

83 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
11 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 9 - Batches: 1 - Items Remaining: 0
25 results available in Pushshift
Total:: Success Rate: 41.67% - Requests: 24 - Batches: 9 - Items Remaining: 0
4 results available in Pushshift
Total:: Success Rate: 66.67% - Requests: 15 - Batches: 3 - Items Remaining: 0
74 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
245 results available in Pushshift
Total:: Success Rate: 66.67% - Requests: 15 - Batches: 2 - Items Remaining: 0
15 results available in Pushshift
Total:: Success Rate: 47.37% - Requests: 19 - Batches: 3 - Items Remaining: 0
23 results available in Pushshift
Total:: Success Rate: 50.00% - Requests: 20 - Batches: 2 - Items Remaining: 0
66 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Re

36 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
24 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 8 - Batches: 1 - Items Remaining: 0
336 results available in Pushshift
Total:: Success Rate: 83.33% - Requests: 12 - Batches: 2 - Items Remaining: 0
5 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 7 - Batches: 1 - Items Remaining: 0
1 results available in Pushshift
Total:: Success Rate: 47.37% - Requests: 19 - Batches: 3 - Items Remaining: 0
173 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
37 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
70 results available in Pushshift
Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 0
178 results available in Pushshift
Total:: Success Rate: 76.92% - Requests: 13 - Batches: 3 - Items 

In [4]:
################################
#The 4th part of the tryptichon:
#Fixing Mistakes:
###################################



#now the same, but for submissions:

def get_all_subs2(before_date,subreddit,time_span = Days,limit = 100000):

  start_time = timeit.default_timer()

  before = before_date
  after = before - time_span
        

  subs=api.search_submissions(subreddit=subreddit, limit=limit, before=before, after=after)

  sub_df = pd.DataFrame(subs)

  name_sub=((datapath%(subreddit+"SUBMISSION_CONTROL")))
  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  clear_output(wait=True)
  end_time=timeit.default_timer()
    
  elapsed = timeit.default_timer() - start_time  
  
    
    

   
    
  print(elapsed/60, "GET_ALL_SUBS")

In [5]:
#Because the rate limit is too low, here a partionioning function of the get_all_coms


def get_all_coms2(before_date,subreddit,partitions_per_month = 5,limit = 100000,time_span = Days ):
    
    start_time = timeit.default_timer()
    time_span = time_span//partitions_per_month
    before = before_date
    for i in range(partitions_per_month):
        after = before - time_span
        subs=api.search_comments(subreddit=subreddit, limit=limit, before=before, after=after)
        before = after
        sub_df = pd.DataFrame(subs)

        name_sub=((datapath%(subreddit+"COMMENT_CONTROL"+str(i))))
        sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))

        clear_output(wait=True)   
              
              

    elapsed = timeit.default_timer() - start_time  




    print(elapsed/60, "get_all_comms_partitioned")


In [11]:
#This here was only necessary, because the limit in the comments_per_sub was set too low. This should be fixed.


#£To do: Shortcels und braincels
subreddit = "darknetmarkets" #This here control, which Subreddit will be completely downloaded
#You don't need to change anything else

Days= 31*24*60*60 #this controls how much back a user has commented to count it as active and study it
Follow_Days = 31*24*60*60

df = pd.read_csv((datapath%(subreddit+"BANISHMENT")))
data=df["created_utc"].sort_values(ascending=False).reset_index(drop=True)
banishment_epoch=data[0]

get_all_subs2(banishment_epoch-Days, subreddit, Days)

#2.2 The Users in the Comments

get_all_coms2(banishment_epoch-Days,subreddit,5, time_span = Days)

#For Convenience, we combine this 5 parts above in one df:

i= 0
b= pd.read_csv(datapath%(subreddit+"COMMENT_CONTROL"+str(i)))

for i in range(4):
    a= pd.read_csv(datapath%(subreddit+"COMMENT_CONTROL"+str(i)))
    b= b.append(a)
    

name_sub=((datapath%(subreddit+"COMMENT_CONTROL")))
b.to_csv(name_sub, header=True, index=False, columns=list(b.axes[1]))

6.161402008333335 get_all_comms_partitioned


In [20]:
#This part is to combine the above

subreddit = "darknetmarkets"
i= 0
b= pd.read_csv(datapath%(subreddit+"COMMENT_CONTROL"+str(i)))

for i in range(4):
    a= pd.read_csv(datapath%(subreddit+"COMMENT_CONTROL"+str(i)))
    b= b.append(a)
    
print(b)
name_sub=((datapath%(subreddit+"COMMENT_CONTROL")))
b.to_csv(name_sub, header=True, index=False, columns=list(b.axes[1]))

                 author author_flair_css_class author_flair_text  \
0            niggsponge                    NaN               NaN   
1            DikTree100                    NaN               NaN   
2           CockAutoBot                    NaN                     
3         AutoModerator                    NaN               NaN   
4     ThePrinceOfFrance                    NaN               NaN   
...                 ...                    ...               ...   
5195        account1189                    new   [Fresh Account]   
5196          Immaloner                  death               NaN   
5197             basjin                    fly               NaN   
5198     ForLol_Serious                    NaN               NaN   
5199          lanikers1                    NaN               NaN   

                                                   body  can_gild  \
0                        Bro I think you figured it out      True   
1     And SWIM is the most interesting man on

In [11]:
def get_all_coms3(before_date,subreddit,partitions_per_month = 5,limit = 100000,time_span = Days ):
    
    start_time = timeit.default_timer()
    time_span = time_span//partitions_per_month
    before = before_date
    for i in range(partitions_per_month):
        after = before - time_span
        subs=api.search_comments(subreddit=subreddit, limit=limit, before=before, after=after)
        before = after
        sub_df = pd.DataFrame(subs)

        name_sub=((datapath%(subreddit+"COMMENT_CONTROL"+str(i))))
        sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))

        clear_output(wait=True)   
              
              

    elapsed = timeit.default_timer() - start_time  




    print(elapsed/60, "get_all_comms_partitioned")


In [13]:
#£To do: Shortcels und braincels
subreddit = "braincels" #This here control, which Subreddit will be completely downloaded
#You don't need to change anything else

Days= 31*24*60*60 #this controls how much back a user has commented to count it as active and study it
Follow_Days = 31*24*60*60

df = pd.read_csv((datapath%(subreddit+"BANISHMENT")))
data=df["created_utc"].sort_values(ascending=False).reset_index(drop=True)
banishment_epoch=data[0]



#2.2 The Users in the Comments

get_all_coms3(banishment_epoch,subreddit,5, time_span = Days)

#For Convenience, we combine this 5 parts above in one df:

i= 0
b= pd.read_csv(datapath%(subreddit+"COMMENT_CONTROL"+str(i)))

for i in range(4):
    a= pd.read_csv(datapath%(subreddit+"COMMENT_CONTROL"+str(i)))
    b= b.append(a)
    

name_sub=((datapath%(subreddit+"COMMENT_complete")))
b.to_csv(name_sub, header=True, index=False, columns=list(b.axes[1]))

In [15]:
########################
#We download here and control if we got all authors form the respective Subreddits
#Here we find out, how many authors we've missed, because of the rate limit of the comments in "shortcels" and "braincels"
subreddit = "braincels"




#Getting Botlist
bad_accounts_raw = pd.read_csv(botlist)

a=0
bad_accounts=[]
#yes, this is the best way I came up with of converting a df into a list
for i in range(len(bad_accounts_raw)):
    bad_accounts.append(bad_accounts_raw.iloc[a,0])
    a=a+1

#here we combine it with the manual bad account list above
bad_accounts.extend(manual_bad_accounts)
    

#Creating List of all unique authors, dropping bad_accounts


df = pd.read_csv(datapath %(subreddit+"COMMENT_complete")) #this is the complete set

df = df[~df.author.isin(bad_accounts)] #this removes all bad accounts contained in our list
    
    
df2 = pd.read_csv(datapath %(subreddit+"COMMENT")) #this is the old set

df2 = df2[~df2.author.isin(bad_accounts)] #see above

df = df[~df.author.isin(df2.author)] #we only want the accounts, which are not in our old list



data= df.groupby(["author"]).size().reset_index(drop=False) #we aggregate all the activity of the accounts and count them.
#This creates a df, which has a axis in which every account is contained and unique


author = list(data.iloc[:,0])#But we actually want a list of all unique authors
print(author)

['-gai-boi-', '066688', '069988244', '0x2412', '246011111', '250kcal', '369oguz', '4070', '666publius666', '69PointstoSlytherin', '778779', '9livesbladeworks', 'ANAL_CAVITIES', 'ATOMIC_ACE_PUGG', 'A_Dull_Vice', 'A_Literal_IT_Cuck', 'AdviceNiided', 'Aehs7', 'Ahsiqa', 'Aitablackcelloser', 'Alaqabi', 'Alcatraz-12', 'AlfaAston03', 'AliBoyyy', 'AlternativaAccount69', 'AltervaneVII', 'AmirosJones', 'AnamuhAF', 'Angelus_lul', 'AnonNo9001', 'Anticrombie233', 'Anyael', 'ArabattaCrumb', 'AraiCRC', 'ArgentinianAlpha', 'Art_Vandelay_In', 'AscendingPizza', 'Asgarnian_Seed', 'AsianScorpio18', 'AskMeAboutMyCornDog', 'Aw3s0m3_K1W1', 'Axubion', 'Bahmanis', 'BazineNetal', 'BeTheBadger', 'Biglenbo14', 'Billyaabob', 'Bingbangboodle21', 'Bingoliums', 'Bio-Mechanic-Man', 'Blackhouseck', 'Blogginginvicecity', 'Blonsomio', 'Bobylein', 'Braincels_account', 'BrazilianSigmma', 'BreakfastBlunt', 'C4ptainoodles', 'CPTfavela', 'CSauceHockey', 'CanadianRADGamer', 'CaptainjustusIII', 'Carl2011', 'Cel9099', 'Ch3fB0yar

In [16]:
#update from second part of python-tryptichon:

#PART THREE: FOLLOWIN THE ACTIVE USERS**************************************************************************

# of all the identified active users of the banned subreddit 
# we want alle the comments and submissions 
# for 31 Days before the ban
# and after the ban.


#First the followed users' submissions before the banning

def get_all_subs_per_auth_before_completer(after_date,before_date,authors,time_span = Follow_Days,limit = 10000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date
  after = before - time_span
  subs = list(range( len(authors)))
#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.
 
  for i in subs:
        subs[i]=api.search_submissions(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%(subreddit+"selected_authors"+"SUBMISSIONS"+"BEFORE_BAN"+"Completer"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    
    
    
  print(completed, "get_all_subs_per_auth_before_finished_above")

        
  

#Secondly the followed users' submissions after the ban
#its basically the same
def get_all_subs_per_auth_after_completer(after_date,before_date,authors,time_span = Follow_Days,limit = 10000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date + Follow_Days+1 #plus one because we don't want overlap
  after = before - time_span
  subs = list(range( len(authors)))
#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.
 
  for i in subs:
        subs[i]=api.search_submissions(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%(subreddit+"selected_authors"+"SUBMISSIONS"+"AFTER_BAN"+"Completer"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    #completed = (%s%s %(str(dt.datetime.fromtimestamp(before))+'   ' +str(len(sub_df)) +'                     '+str(np.round((end_time-start_time)/60,2))+'\n'
    
    
  print(completed, "get_all_subs_per_auth_after_finished_above")

        

#Now the same, but for comments

def get_all_coms_per_auth_before_completer(after_date,before_date,authors,time_span = Follow_Days,limit = 10000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date
  after = before - time_span
  subs = list(range( len(authors)))
#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.
 
  for i in subs:
        subs[i]=api.search_comments(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%(subreddit+"selected_authors"+"COMMENTS"+"BEFORE_BAN"+"Completer"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    
    
    
  print(completed,"get_all_coms_per_auth_before_finished_above")


#now comments after the ban:

def get_all_coms_per_auth_after_completer(after_date,before_date,authors,time_span = Follow_Days,limit = 10000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date + Follow_Days+1 #plus one because we don't want overlap
  after = before - time_span
  subs = list(range( len(authors)))
#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.
 
  for i in subs:
        subs[i]=api.search_comments(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%(subreddit+"selected_authors"+"COMMENTS"+"AFTER_BAN"+"Completer"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    #completed = (%s%s %(str(dt.datetime.fromtimestamp(before))+'   ' +str(len(sub_df)) +'                     '+str(np.round((end_time-start_time)/60,2))+'\n'
    
    
  print(completed, "get_all_coms_per_auth_after_finished")
  if sub_len == 100000:
        print("theres too much Data, Information has been lost !!!11!11 comments_user_after************************************************************************************************************")



In [17]:
#3. Following our identified Users**********************************************************
#3.1 comments before ban

get_all_coms_per_auth_before_completer(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 10000)

#3.2 comments after ban

get_all_coms_per_auth_after_completer(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 100000)

#3.3 Submissions before ban

get_all_subs_per_auth_before_completer(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 100000)


#3.4 Submissions after ban


get_all_subs_per_auth_after_completer(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 100000)

print("If This shows, you really did it Claudio!!!")

  after date            # of subs        time in minutes

2019-09-30 20:06:17        3224           74.99
 get_all_subs_per_auth_after_finished_above
If This shows, you really did it Claudio!!!


In [2]:
#### We follow incels
#new definition

def get_all_subs_per_auth_long_exposure(after_date,before_date,authors,time_span = Follow_Days,limit = 2000):
  completed = '  after date            # of subs        time in minutes\n'
  start_time = timeit.default_timer()

  before = before_date
  after = before - time_span
  subs = list(range( len(authors)))
#we want to feed the function the authors in packs of 100. It has been packed before in lists in lists of 100. 
#And we call it by the first list, the second list the i-th list usw.
 
  for i in subs:
        subs[i]=api.search_submissions(author=authors[i], limit=limit, before=before, after=after)
        

  sub_df = pd.DataFrame(chain.from_iterable((subs))) #this is that we don't get list of lists in the df, otherwise it destroys the structure

  name_sub=(datapath%("INCEL_ACCOUNTS"+"selected_authors"+"SUBMISSIONS"+"LONGEXPOSURE+DifferentDAte"))

  sub_df.to_csv(name_sub, header=True, index=False, columns=list(sub_df.axes[1]))
  

  clear_output(wait=True)
  end_time=timeit.default_timer()
    
    
  sub_len = len(sub_df)
    
    
  completed=completed + ('\n%s      %6d        %8.2f\n'%(str(dt.datetime.fromtimestamp(after)),sub_len,((end_time-start_time)/60)))
    
    

    
    
    
  print(completed, "long exposure completed")

In [5]:

############ We follow incels

#1.2 extracting the banishment-time

df = pd.read_csv((datapath%("Braincels"+"BANISHMENT")))
data=df["created_utc"].sort_values(ascending=False).reset_index(drop=True)
banishment_epoch_braincels=data[0]

df = pd.read_csv((datapath%("incelswithouthate"+"BANISHMENT")))
data=df["created_utc"].sort_values(ascending=False).reset_index(drop=True)
banishment_epoch_incelswithouthate=data[0]


#Getting Botlist
bad_accounts_raw = pd.read_csv(botlist)

a=0
bad_accounts=[]
#yes, this is the best way I came up with of converting a df into a list
for i in range(len(bad_accounts_raw)):
    bad_accounts.append(bad_accounts_raw.iloc[a,0])
    a=a+1

#here we combine it with the manual bad account list above
bad_accounts.extend(manual_bad_accounts)
    

#Creating List of all unique authors, dropping bad_accounts


df = pd.read_csv(datapath %("Braincels"+"SUBMISSION"))

df = df[~df.author.isin(bad_accounts)] #this removes all bad accounts contained in our list
    



data= df.groupby(["author"]).size().reset_index(drop=False) #we aggregate all the activity of the accounts and count them.
#This creates a df, which has a axis in which every account is contained and unique


author = list(data.iloc[:,0])#But we actually want a list of all unique authors


#3.1 comments before ban

Follow_days= banishment_epoch_incelswithouthate - banishment_epoch_braincels+Days #That means all the time between the two bans plus 1 month before the first  ban 


#author_sample=random.sample(author,100)

get_all_subs_per_auth_long_exposure(banishment_epoch_braincels, banishment_epoch_incelswithouthate, author, Follow_days)


  after date            # of subs        time in minutes

2019-08-30 20:06:16      137821          589.83
 long exposure completed


In [ ]:
## For the experiment we want control group of the submissions 1 month before the treatment-group


#1.2 extracting the banishment-time

df = pd.read_csv((datapath%(subreddit+"BANISHMENT")))
data=df["created_utc"].sort_values(ascending=False).reset_index(drop=True)
banishment_epoch=data[0]-Days #here we have "-Days" which is different than in the treatment group. everything will be
#pushed a month back


get_all_subs_control(banishment_epoch-Days, banishment_epoch, subreddit, Days)

#2.3 Creating botlist and other bad_accountsnames

#Getting Botlist
bad_accounts_raw = pd.read_csv(botlist)

a=0
bad_accounts=[]
#yes, this is the best way I came up with of converting a df into a list
for i in range(len(bad_accounts_raw)):
    bad_accounts.append(bad_accounts_raw.iloc[a,0])
    a=a+1

#here we combine it with the manual bad account list above
bad_accounts.extend(manual_bad_accounts)
    

#Creating List of all unique authors, dropping bad_accounts



df = pd.read_csv(datapath %((subreddit+"SUBMISSION_CONTROLGROUP")))

df = df[~df.author.isin(bad_accounts)] #this removes all bad accounts contained in our list
    
data= df.groupby(["author"]).size().reset_index(drop=False) #we aggregate all the activity of the accounts and count them.
#This creates a df, which has a axis in which every account is contained and unique


author = list(data.iloc[:,0])#But we actually want a list of all unique authors
print(len(author))

#3. Following our identified Users**********************************************************

#3.1 Submissions before ban

get_all_subs_per_auth_before_control(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 10000)


#3.2 Submissions after ban


get_all_subs_per_auth_after_control(banishment_epoch-Follow_Days,banishment_epoch,author,time_span = Follow_Days,limit = 10000)

print("If This shows, you really did it Claudio!!!")
    

Not all PushShift shards are active. Query results may be incomplete.


1503 results available in Pushshift


In [3]:
# This would be the control of the controlgroup (but pushsift is down for more than a week :( )

df = pd.read_csv((datapath%(subreddit+"BANISHMENT")))
data=df["created_utc"].sort_values(ascending=False).reset_index(drop=True)
banishment_epoch=data[0]-Days #here we have "-Days" which is different than in the treatment group. everything will be
#pushed a month back

get_all_subs_control_of_control(banishment_epoch-Days, banishment_epoch, subreddit, Days)

Not all PushShift shards are active. Query results may be incomplete.


1503 results available in Pushshift


Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.


Total:: Success Rate: 100.00% - Requests: 20 - Batches: 2 - Items Remaining: 0
0.3814726883333333 GET_ALL_SUBS
